# Setup

In [2]:
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import date
import plotly.io as pio
pio.templates.default = "plotly"

# Viz 8

In [73]:
df_a = pd.read_excel('Table3A.xlsx')

df_a['Unemployed_Actual'] = df_a['Unemployed']*1000
df_a['No Grade Completed'] = (df_a['No Grade Completed']/100)*df_a['Unemployed_Actual']
df_a['Elementary'] = (df_a['Elementary']/100)*df_a['Unemployed_Actual']
df_a['Junior Highschool'] = (df_a['Junior Highschool']/100)*df_a['Unemployed_Actual']
df_a['Senior Highschool'] = (df_a['Senior Highschool']/100)*df_a['Unemployed_Actual']
df_a['Post Secondary'] = (df_a['Post Secondary']/100)*df_a['Unemployed_Actual']
df_a['College'] = (df_a['College']/100)*df_a['Unemployed_Actual']

In [74]:
viz8 = go.Figure()

education_levels = ['No Grade Completed','Elementary', 'Junior Highschool', 'Senior Highschool', 'Post Secondary', 'College']
colors = ['#9a62a4','#9a62a4','#9a62a4','#9a62a4','#9a62a4','#9a62a4']

for i, edu_level in enumerate(education_levels):
  viz8.add_trace(go.Bar(
      y=[edu_level],
      x=[df_a[edu_level].iloc[0]],
      marker_color=colors[i],
      text=[str(int(df_a[edu_level].iloc[0]))],
      textposition='outside',
      hovertemplate='No. of Unemployed: %{x:.2f}<extra></extra>',
      orientation='h'
  ))

viz8.update_layout(
  title='Number of Unemployed Workers by Highest Educational Attainment (2021)',
  xaxis_title='Number of Unemployed Workers',
  yaxis_title='Highest Educational Attainment',
  width=900,
  height=600,
  margin=dict(l=150, r=50, t=100, b=50),
  showlegend=False,
  hoverlabel=dict(namelength=0),
  xaxis=dict(range=[0, 1.7e6])

)
viz8.show()

# Viz 9

In [81]:
# Importing
df1 = pd.read_excel('Viz9.xlsx', sheet_name='Sheet1', usecols=['Region', 'Total_Pop_15_Years_Old _and_Over_E', 'Labor_Force_Participation_Rate', 'Employment_Rate', 'Unemployment_Rate'])
df2 = pd.read_excel('Viz9.xlsx', sheet_name='Sheet2', usecols=['Region', 'Income', 'Expenditure'])
df3 = pd.read_excel('Viz9.xlsx', sheet_name='Sheet3', usecols=['Region', 'AvgIncome', 'AvgExpenditure', 'GrowthRate_AveIncome', 'GrowthRate_AveExpenditure'])

df_merged = pd.merge(df1, df2, on='Region')
df_merged = pd.merge(df_merged, df3, on='Region')

df_merged = df_merged.rename(columns={'Labor_Force_Participation_Rate': 'Labor Force Participation Rate',
                                      'Employment_Rate': 'Employment Rate',
                                      'Unemployment_Rate': 'Unemployment Rate',
                                      'AvgIncome': 'Average Income',
                                      'AvgExpenditure': 'Average Expenditure',
                                      'GrowthRate_AveIncome':'Average Income Growth Rate',
                                      'GrowthRate_AveExpenditure':'Average Expenditure Growth Rate',
                                      'Total_Pop_15_Years_Old _and_Over_E':'Working Age Population'})

df_merged = df_merged.replace({'Region I':'REGION I - ILOCOS REGION',
                                      'Region II':'REGION II - CAGAYAN VALLEY',
                                      'Region IIi':'REGION III - CENTRAL LUZON',
                                      'Region IVA':'REGION IVA - CALABARZON',
                                      'Region V':'REGION V - BICOL REGION',
                                      'CARAGA':'REGION XIII - CARAGA',
                                      'BANGSAMORO AUTONOMOUS REGION IN MUSLIM MINDANAO':'BARMM',
                                      'Mimaropa':'MIMAROPA'})

In [82]:
x_var = ['Labor Force Participation Rate','Employment Rate','Unemployment Rate','Average Income','Average Expenditure','Average Income Growth Rate','Average Expenditure Growth Rate','Working Age Population']
colorscale = [[0, '#fcbb81'], [0.5, '#ffffff'], [1, '#9a62a4']]

# Create a heatmap trace
trace = go.Heatmap(
  x=x_var,
  y=x_var,
  z=df_merged[x_var].corr(),
  colorscale=colorscale,
  zmin=-1,
  zmax=1,
  colorbar=dict(title='Correlation'),
  hovertemplate='%{z:.2f}<extra></extra>',
)

# Set the layout of the heatmap
layout = go.Layout(
  title='Correlation Matrix',
  width=1000,
  height=800,
  margin=dict(l=150, r=50, t=100, b=50),
  xaxis=dict(title='Variables'),
  yaxis=dict(title='Variables'),
)

# Create a figure and add the heatmap trace to it
viz9 = go.Figure(data=[trace], layout=layout)

# Viz 10

In [80]:
# Step 1: Define default x and y values
default_x = "Average Income"  # replace with your desired column name for x-axis
default_y = "Average Expenditure"  # replace with your desired column name for y-axis

# Step 2: Create a function to generate the default scatterplot
if df_merged[default_x].corr(df_merged[default_y]) > 0:
    color = '#9a62a4'
else:
    color = '#fcbb81'

viz10 = px.scatter(df_merged, x=default_x, y=default_y, color="Region",
                 hover_name="Region", color_discrete_sequence=[color],
                 width=1000, height=500, size_max=8)

viz10.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
    legend=dict(title=None),
    font_family="Arial",
    plot_bgcolor="#f7f7f7")


# Insights

In [79]:
# Creating the app
page = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])


page.layout = dbc.Container([

    
    # Header

    
    html.Nav(
    html.Ul([
        html.Li([
            html.A('Home', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Understanding Poverty', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Zooming in on Employment', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item'),
        html.Li([
            html.A('Finding Opportunities', href='#', className='nav-link', style={'border': '1px solid grey'})
        ], className='nav-item')
    ], className='nav'),

),
  
  # Line Breaks

     html.Div([
          html.Br(), html.Br(), html.Br(), html.Br(),  html.Br(), 
     ]),
    

    
    # Header 3 
    html.Section([
        html.H2("Finding Opportunities", className="text-center mb-3"),
        html.H5("", className="text-center mb-5"),
    ]),

    
    # Insight 9
    html.Section([
        

        html.Div([
            dcc.Graph(id="viz8", figure=viz8, style={'width': '18px'}, className = ""),
            dcc.Graph(id="viz9", figure=viz9, style={'width': '18px'}, className = ""),
            dcc.Graph(id='viz10', animate=False),
            html.H6("Although labor force growth is accompanied by a rise in both average income and expenditures, the latter is more elastic to changes...", className="mb-5"),

            html.P("This debunks the misconception that higher working age population, and in turn, greater job competition stifles purchasing power, but the 0.64 and 0.70 correlation coefficients show that the market adjusts upwards. These financial gains likely stem from the additional productivity and volume consumption brought about by a larger employee base. It is also possible that higher competition fuels more efforts towards job readiness among workers that fuels productivity once they are hired. Regardless, there is still a gap between the rise in expenditures and income and not everyone will be prepared to face this. Especially for first-time workers, there is a need for not just financial literacy, but negotiation skills to self-advocate during the hiring process for skill-appropriate compensation.", className="mb-5"),

            html.H6("... but this trend is expected to dampen as expenditure growth declines more quickly than income growth.", className="mb-5"),

            html.P("Both metrics show negative correlations of -0.56 and -0.60, respectively, with working age population.", className="mb-5"),
            
        


       ], className="page3"
    ),
  

    ]),

])


@page.callback(
    Output("viz10", "figure"),
    [Input("viz9", "clickData")]
)

def update_viz10(click_data):
    # Define default x and y values
    default_x = "Average Income"  # replace with your desired column name for x-axis
    default_y = "Average Expenditure"  # replace with your desired column name for y-axis

    if click_data is None:
        x, y = default_x, default_y
    else:
        x = click_data['points'][0]['x']
        y = click_data['points'][0]['y']

    if df_merged[x].corr(df_merged[y]) > 0:
        color = '#9a62a4'
    else:
        color = '#fcbb81'

    fig = px.scatter(df_merged, x=x, y=y, color="Region",
                     hover_name="Region", color_discrete_sequence=[color],
                     width=1000, height=500, size_max=8)

    fig.update_layout(
        margin=dict(l=20, r=20, t=20, b=20),
        legend=dict(title=None),
        font_family="Arial",
        plot_bgcolor="#f7f7f7")

    return fig


if __name__ == '__main__':
  page.run_server(debug=True, port = 8033)

Dash app running on http://127.0.0.1:8033/
